In [10]:
import os
import pandas as pd
import numpy as np
import mlflow
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

DATA_DIR = "/usr/mlflow/data"

anime = pd.read_csv(os.path.join(DATA_DIR, "anime_clean.csv"))
ratings_train = pd.read_csv(os.path.join(DATA_DIR, "ratings_train.csv"))
ratings_test = pd.read_csv(os.path.join(DATA_DIR, "ratings_test.csv"))

print("Anime:", anime.shape)
print("Train:", ratings_train.shape)
print("Test:", ratings_test.shape)

Anime: (12294, 7)
Train: (633755, 3)
Test: (316831, 3)


In [11]:
# 建立 user-item 矩陣
user_item_matrix = ratings_train.pivot_table(
    index="user_id", columns="anime_id", values="rating"
).fillna(0)

knn = NearestNeighbors(metric="cosine", algorithm="brute", n_neighbors=6, n_jobs=-1)
knn.fit(user_item_matrix)

def recommend_user_based(user_id, top_n=10):
    if user_id not in user_item_matrix.index:
        return pd.DataFrame(columns=["anime_id", "name", "genre"])
    
    user_vector = user_item_matrix.loc[[user_id]]
    distances, indices = knn.kneighbors(user_vector, n_neighbors=6)
    neighbor_ids = user_item_matrix.index[indices.flatten()[1:]]  # 排除自己
    
    neighbor_ratings = user_item_matrix.loc[neighbor_ids]
    mean_scores = neighbor_ratings.mean().sort_values(ascending=False)
    
    seen = user_item_matrix.loc[user_id]
    seen = seen[seen > 0].index
    recommendations = mean_scores.drop(seen).head(top_n)
    
    return anime[anime["anime_id"].isin(recommendations.index)][["anime_id","name","genre"]]

In [12]:
anime["text"] = anime["genre"].fillna("") + " " + anime["type"].fillna("")

tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(anime["text"])
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
indices = pd.Series(anime.index, index=anime["anime_id"]).drop_duplicates()

def recommend_item_based(user_id, top_n=10):
    user_ratings = ratings_train[ratings_train["user_id"] == user_id]
    liked = user_ratings[user_ratings["rating"] > 7]["anime_id"].tolist()
    
    if len(liked) == 0:
        return pd.DataFrame(columns=["anime_id","name"])
    
    sim_scores = np.zeros(cosine_sim.shape[0])
    for anime_id in liked:
        if anime_id in indices:
            idx = indices[anime_id]
            sim_scores += cosine_sim[idx]
    
    sim_scores = sim_scores / len(liked)
    sim_indices = sim_scores.argsort()[::-1]
    
    seen = set(user_ratings["anime_id"])
    rec_ids = [anime.loc[i, "anime_id"] for i in sim_indices if anime.loc[i, "anime_id"] not in seen][:top_n]
    
    return anime[anime["anime_id"].isin(rec_ids)][["anime_id","name","genre"]]

In [13]:
def precision_recall_at_k(user_id, model_func, k=10):
    recs = model_func(user_id, top_n=k)
    if recs.empty:
        return np.nan, np.nan
    
    user_test = ratings_test[ratings_test["user_id"] == user_id]
    liked = set(user_test[user_test["rating"] > 7]["anime_id"])
    if len(liked) == 0:
        return np.nan, np.nan
    
    hit = len(set(recs["anime_id"]) & liked)
    
    precision = hit / k
    recall = hit / len(liked)
    return precision, recall

In [14]:
sample_users = np.random.choice(ratings_train["user_id"].unique(), 300, replace=False)

def evaluate_model(model_func, name):
    precisions, recalls = [], []
    for u in sample_users:
        p, r = precision_recall_at_k(u, model_func, 10)
        if not np.isnan(p): precisions.append(p)
        if not np.isnan(r): recalls.append(r)
    return {
        "model": name,
        "precision@10": np.mean(precisions),
        "recall@10": np.mean(recalls)
    }

results = []
results.append(evaluate_model(recommend_user_based, "user_based_cf"))
results.append(evaluate_model(recommend_item_based, "item_based_tfidf"))

df_compare = pd.DataFrame(results)
print(df_compare)

              model  precision@10  recall@10
0     user_based_cf      0.004608   0.015916
1  item_based_tfidf      0.005164   0.016523


In [16]:
report_users = ratings_test["user_id"].drop_duplicates().sample(3, random_state=42)

records = []
for u in report_users:
    liked = ratings_test[(ratings_test["user_id"] == u) & (ratings_test["rating"] > 7)]
    liked_names = anime[anime["anime_id"].isin(liked["anime_id"])]["name"].tolist()
    
    recs_u = recommend_user_based(u, top_n=10)["name"].tolist()
    recs_i = recommend_item_based(u, top_n=10)["name"].tolist()
    
    records.append({
        "user_id": u,
        "liked_in_test": liked_names,
        "user_based_recommend": recs_u,
        "item_based_recommend": recs_i
    })

df_examples = pd.DataFrame(records)
print(df_examples)

   user_id                                      liked_in_test  \
0    43880                                           [Naruto]   
1    12909  [Binbougami ga!, Dungeon ni Deai wo Motomeru n...   
2    69001  [Higashi no Eden Movie I: The King of Eden, K,...   

                                user_based_recommend  \
0  [Cowboy Bebop, Towa no Quon 6: Towa no Quon, T...   
1  [Cowboy Bebop, Death Note, Dragon Ball Z, FLCL...   
2  [Kokoro Connect: Michi Random, Mawaru Penguind...   

                                item_based_recommend  
0  [Gungrave, Guilty Crown, Kurau Phantom Memory,...  
1  [Wolf&#039;s Rain, Bungou Stray Dogs, GetBacke...  
2  [High School DxD BorN, Trinity Seven, Omamori ...  


In [17]:
mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI", "http://mlflow:5000"))
mlflow.set_experiment("anime-recommender-report")

# 存 CSV
df_compare.to_csv("day9_compare_metrics.csv", index=False)
df_examples.to_csv("day9_recommend_examples.csv", index=False)

with mlflow.start_run(run_name="day9_model_report"):
    # log metrics
    for row in results:
        mlflow.log_metric(f"{row['model']}_precision_at_10", row["precision@10"])
        mlflow.log_metric(f"{row['model']}_recall_at_10", row["recall@10"])
    
    # log artifacts
    mlflow.log_artifact("day9_compare_metrics.csv", artifact_path="reports")
    mlflow.log_artifact("day9_recommend_examples.csv", artifact_path="reports")

2025/09/21 16:24:50 INFO mlflow.tracking.fluent: Experiment with name 'anime-recommender-report' does not exist. Creating a new experiment.
